**Секція 1. Логістична регресія з нуля.**

Будемо крок за кроком будувати модель лог регресії з нуля для передбачення, чи буде врожай більше за 80 яблук (задача подібна до лекційної, але на класифікацію).

Давайте нагадаємо основні формули для логістичної регресії.

### Функція гіпотези - обчислення передбачення у логістичній регресії:

$$
\hat{y} = \sigma(x W^T + b) = \frac{1}{1 + e^{-(x W^T + b)}}
$$

Де:
- $ \hat{y} $ — це ймовірність "позитивного" класу.
- $ x $ — це вектор (або матриця для набору прикладів) вхідних даних.
- $ W $ — це вектор (або матриця) вагових коефіцієнтів моделі.
- $ b $ — це зміщення (bias).
- $ \sigma(z) $ — це сигмоїдна функція активації.

### Як обчислюється сигмоїдна функція:

Сигмоїдна функція $ \sigma(z) $ має вигляд:

$$
\sigma(z) = \frac{1}{1 + e^{-z}}
$$

Ця функція перетворює будь-яке дійсне значення $ z $ в інтервал від 0 до 1, що дозволяє інтерпретувати вихід як ймовірність для логістичної регресії.

### Формула функції втрат для логістичної регресії (бінарна крос-ентропія):

Функція втрат крос-ентропії оцінює, наскільки добре модель передбачає класи, порівнюючи передбачені ймовірності $ \hat{y} $ із справжніми мітками $ y $. Формула наступна:

$$
L(y, \hat{y}) = - \left[ y \cdot \log(\hat{y}) + (1 - y) \cdot \log(1 - \hat{y}) \right]
$$

Де:
- $ y $ — це справжнє значення (мітка класу, 0 або 1).
- $ \hat{y} $ — це передбачене значення (ймовірність).



1.
Тут вже наведений код для ініціювання набору даних в форматі numpy. Перетворіть `inputs`, `targets` на `torch` тензори. Виведіть результат на екран.

In [68]:
import torch
import numpy as np

In [69]:
# Вхідні дані (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43],
                   [91, 88, 64],
                   [87, 134, 58],
                   [102, 43, 37],
                   [69, 96, 70]], dtype='float32')

# Таргети (apples > 80)
targets = np.array([[0],
                    [1],
                    [1],
                    [0],
                    [1]], dtype='float32')

In [70]:
#перетворення на torch тензори
inputs = torch.tensor(inputs, dtype=torch.float32)
targets = torch.tensor(targets, dtype=torch.float32)
print(inputs)
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[0.],
        [1.],
        [1.],
        [0.],
        [1.]])


2. Ініціюйте ваги `w`, `b` для моделі логістичної регресії потрібної форми зважаючи на розмірності даних випадковими значеннями з нормального розподілу. Лишаю тут код для фіксації `random_seed`.

In [71]:
torch.random.manual_seed(1)

In [72]:
# Ініціалізація ваг нормальним розподілом
w = torch.randn(1, 3, requires_grad=True )
b = torch.randn(1, requires_grad=True)

print("w:")
print(w)

print("\nb:")
print(b)

w:
tensor([[0.6614, 0.2669, 0.0617]], requires_grad=True)

b:
tensor([0.6213], requires_grad=True)


3. Напишіть функцію `model`, яка буде обчислювати функцію гіпотези в логістичній регресії і дозволяти робити передбачення на основі введеного рядка даних і коефіцієнтів в змінних `w`, `b`.

  **Важливий момент**, що функція `model` робить обчислення на `torch.tensors`, тож для математичних обчислень використовуємо фукнціонал `torch`, наприклад:
  - обчсилення $e^x$: `torch.exp(x)`
  - обчсилення $log(x)$: `torch.log(x)`
  - обчислення середнього значення вектору `x`: `torch.mean(x)`

  Використайте функцію `model` для обчислення передбачень з поточними значеннями `w`, `b`.Виведіть результат обчислень на екран.

  Проаналізуйте передбачення. Чи не викликають вони у вас підозр? І якщо викликають, то чим це може бути зумовлено?

In [73]:
def model(x, w, b):
    linear = x @ w.t() + b      # [5,3] @ [3,1] -[5,1]
    return 1 / (1 + torch.exp(-linear))

In [74]:
preds = model(inputs, w, b)

print("\nPreds:")
print(preds)
print("\nTargets:")
print(targets)


Preds:
tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.]], grad_fn=<MulBackward0>)

Targets:
tensor([[0.],
        [1.],
        [1.],
        [0.],
        [1.]])


Як на мене підозри поки немає. Модель передбачає врожай, що загалом є логічним, одразу була б підозра, якщо врожай би не передбачався.

4. Напишіть функцію `binary_cross_entropy`, яка приймає на вхід передбачення моделі `predicted_probs` та справжні мітки в даних `true_labels` і обчислює значення втрат (loss)  за формулою бінарної крос-ентропії для кожного екземпляра та вертає середні втрати по всьому набору даних.
  Використайте функцію `binary_cross_entropy` для обчислення втрат для поточних передбачень моделі.

In [75]:
def binary_cross_entropy(predicted_probs,true_labels):
  loss = - (true_labels* torch.log(predicted_probs) +
   (1 - true_labels)*torch.log(1 - predicted_probs))
  return loss.mean()

In [76]:
loss = binary_cross_entropy(preds,targets)
loss

tensor(nan, grad_fn=<MeanBackward0>)

Ціль binary_cross_entropy дати нам оцінку від 0 до 1, де 0 це модель до якої ми прагнемо. Так як в попередній моделі в мене прогноз 1, тобто врожайність є, то і binary_cross_entropy дає нам нан, як 0.

5. Зробіть зворотнє поширення помилки і виведіть градієнти за параметрами `w`, `b`. Проаналізуйте їх значення. Як гадаєте, чому вони саме такі?

In [77]:
# Обчислимо gradients
loss.backward()

In [78]:
# Градієнти вагів
print(w)
print(w.grad)

tensor([[0.6614, 0.2669, 0.0617]], requires_grad=True)
tensor([[nan, nan, nan]])


In [79]:
# Градієнти вагів
print(b)
print(b.grad)

tensor([0.6213], requires_grad=True)
tensor([nan])


В нас елемент градієнта позитивний та спостерігаємо, що зменшення значення (та = 0) елемента трохи збільшить втрати

**Що сталось?**

В цій задачі, коли ми ініціювали значення випадковими значеннями з нормального розподілу, насправді ці значення не були дуже гарними стартовими значеннями і привели до того, що градієнти стали дуже малими або навіть рівними нулю (це призводить до того, що градієнти "зникають"), і відповідно при оновленні ваг у нас не буде нічого змінюватись. Це називається `gradient vanishing`. Це відбувається через **насичення сигмоїдної функції активації.**

У нашій задачі ми використовуємо сигмоїдну функцію активації, яка має такий вигляд:

   $$
   \sigma(z) = \frac{1}{1 + e^{-z}}
   $$


Коли значення $z$ дуже велике або дуже мале, сигмоїдна функція починає "насичуватись". Це означає, що для великих позитивних $z$ сигмоїда наближається до 1, а для великих негативних — до 0. В цих діапазонах градієнти починають стрімко зменшуватись і наближаються до нуля (бо градієнт - це похідна, похідна на проміжку функції, де вона паралельна осі ОХ, дорівнює 0), що робить оновлення ваг неможливим.

![](https://editor.analyticsvidhya.com/uploads/27889vaegp.png)

У логістичній регресії $ z = x \cdot w + b $. Якщо ваги $w, b$ - великі, значення $z$ також буде великим, і сигмоїда перейде в насичену область, де градієнти дуже малі.

Саме це сталося в нашій задачі, де великі випадкові значення ваг викликали насичення сигмоїдної функції. Це в свою чергу призводить до того, що під час зворотного поширення помилки (backpropagation) модель оновлює ваги дуже повільно або зовсім не оновлює. Це називається проблемою **зникнення градієнтів** (gradient vanishing problem).

**Що ж робити?**
Ініціювати ваги маленькими значеннями навколо нуля. Наприклад ми можемо просто в існуючій ініціалізації ваги розділити на 1000. Можна також використати інший спосіб ініціалізації вагів - інформація про це [тут](https://www.geeksforgeeks.org/initialize-weights-in-pytorch/).

Як це робити - показую нижче. **Виконайте код та знову обчисліть передбачення, лосс і виведіть градієнти.**

А я пишу пояснення, чому просто не зробити

```
w = torch.randn(1, 3, requires_grad=True)/1000
b = torch.randn(1, requires_grad=True)/1000
```

Нам потрібно, аби тензори вагів були листовими (leaf tensors).

1. **Що таке листовий тензор**
Листовий тензор — це тензор, який був створений користувачем безпосередньо і з якого починається обчислювальний граф. Якщо такий тензор має `requires_grad=True`, PyTorch буде відслідковувати всі операції, виконані над ним, щоб правильно обчислювати градієнти під час навчання.

2. **Чому ми використовуємо `w.data` замість звичайних операцій**
Якщо ми просто виконали б операції, такі як `(w - 0.5) / 100`, ми б отримали **новий тензор**, який вже не був би листовим тензором, оскільки ці операції створюють **новий** тензор, а не модифікують існуючий.

  Проте, щоб залишити наші тензори ваги `w` та зміщення `b` листовими і продовжити можливість відстеження градієнтів під час тренування, ми використовуємо атрибут `.data`. Цей атрибут дозволяє **виконувати операції in-place (прямо на існуючому тензорі)** без зміни самого об'єкта тензора. Отже, тензор залишається листовим, і PyTorch може коректно обчислювати його градієнти.

3. **Чому важливо залишити тензор листовим**
Якщо тензор більше не є листовим (наприклад, через проведення операцій, що створюють нові тензори), ви не зможете отримати градієнти за допомогою `w.grad` чи `b.grad` після виклику `loss.backward()`. Це може призвести до втрати можливості оновлення параметрів під час тренування моделі. В нашому випадку ми хочемо, щоб тензори `w` та `b` накопичували градієнти, тому вони повинні залишатись листовими.

**Висновок:**
Ми використовуємо `.data`, щоб виконати операції зміни значень на ваги і зміщення **in-place**, залишаючи їх листовими тензорами, які можуть накопичувати градієнти під час навчання. Це дозволяє коректно працювати механізму зворотного поширення помилки (backpropagation) і оновлювати ваги моделі.

5. Виконайте код та знову обчисліть передбачення, лосс і знайдіть градієнти та виведіть всі ці тензори на екран.

In [80]:
torch.random.manual_seed(1)
w = torch.randn(1, 3, requires_grad=True)  # Листовий тензор
b = torch.randn(1, requires_grad=True)     # Листовий тензор

# in-place операції
w.data = w.data / 1000
b.data = b.data / 1000

In [82]:
preds_v2 = model(inputs, w, b)

print("\nPreds_v2:")
print(preds_v2)
print("\nTargets:")
print(targets)


Preds_v2:
tensor([[0.5174],
        [0.5220],
        [0.5244],
        [0.5204],
        [0.5190]], grad_fn=<MulBackward0>)

Targets:
tensor([[0.],
        [1.],
        [1.],
        [0.],
        [1.]])


In [84]:
loss_v2 = binary_cross_entropy(preds_v2,targets)
loss_v2

tensor(0.6829, grad_fn=<MeanBackward0>)

In [85]:
# Обчислимо gradients
loss_v2.backward()

In [86]:
# Градієнти вагів
print(w)
print(w.grad)

tensor([[6.6135e-04, 2.6692e-04, 6.1677e-05]], requires_grad=True)
tensor([[ -5.4417, -18.9853, -10.0682]])


In [87]:
# Градієнти вагів
print(b)
print(b.grad)

tensor([0.0006], requires_grad=True)
tensor([-0.0794])


6. Напишіть алгоритм градієнтного спуску, який буде навчати модель з використанням написаних раніше функцій і виконуючи оновлення ваг. Алгоритм має включати наступні кроки:

  1. Генерація прогнозів
  2. Обчислення втрат
  3. Обчислення градієнтів (gradients) loss-фукнції відносно ваг і зсувів
  4. Налаштування ваг шляхом віднімання невеликої величини, пропорційної градієнту (`learning_rate` домножений на градієнт)
  5. Скидання градієнтів на нуль

Виконайте градієнтний спуск протягом 1000 епох, обчисліть фінальні передбачення і проаналізуйте, чи вони точні?

In [96]:
#поточна конвенція для маленького значення кроку градієнтного спуску
learning_rate = 1e-4

In [95]:
for epoch in range(1000):
#генерація прогнозів
    preds_v3 = model(inputs, w, b)

#Обчислення втрат
    loss_v3 = binary_cross_entropy(preds_v3,targets)

#Обчислення градієнтів (gradients) loss-фукнції відносно ваг і зсувів
    loss_v3.backward()

#Налаштування ваг шляхом віднімання невеликої величини, пропорційної градієнту (learning_rate домножений на градієнт)
    with torch.no_grad():
      w -= learning_rate * w.grad
      b -= learning_rate * b.grad
#Скидання градієнтів на нуль
    w.grad.zero_()
    b.grad.zero_()

    if (epoch+1) % 100 == 0:
        print(f"Epoch {epoch+1}, Loss: {loss_v3.item():.4f}")

Epoch 100, Loss: 0.1957
Epoch 200, Loss: 0.1913
Epoch 300, Loss: 0.1872
Epoch 400, Loss: 0.1833
Epoch 500, Loss: 0.1796
Epoch 600, Loss: 0.1762
Epoch 700, Loss: 0.1728
Epoch 800, Loss: 0.1696
Epoch 900, Loss: 0.1665
Epoch 1000, Loss: 0.1636


З кожною епохою втрати падають/зменшуються, головне що воно йде потихеньку і з насутпними епохами скоріш за все він також буде падати/зменшуютьсь

**Секція 2. Створення лог регресії з використанням функціоналу `torch.nn`.**

Давайте повторно реалізуємо ту ж модель, використовуючи деякі вбудовані функції та класи з PyTorch.

Даних у нас буде побільше - тож, визначаємо нові масиви.

In [131]:
# Вхідні дані (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43],
                   [91, 88, 64],
                   [87, 134, 58],
                   [102, 43, 37],
                   [69, 96, 70],
                   [73, 67, 43],
                   [91, 88, 64],
                   [87, 134, 58],
                   [102, 43, 37],
                   [69, 96, 70],
                   [73, 67, 43],
                   [91, 88, 64],
                   [87, 134, 58],
                   [102, 43, 37],
                   [69, 96, 70]], dtype='float32')

# Таргети (apples > 80)
targets = np.array([[0],
                    [1],
                    [1],
                    [0],
                    [1],
                    [0],
                    [1],
                    [1],
                    [0],
                    [1],
                    [0],
                    [1],
                    [1],
                    [0],
                    [1]], dtype='float32')

7. Завантажте вхідні дані та мітки в PyTorch тензори та з них створіть датасет, який поєднує вхідні дані з мітками, використовуючи клас `TensorDataset`. Виведіть перші 3 елементи в датасеті.



In [132]:
# Імпортуємо модуль nn
import torch.nn as nn

# Імпортуємо tensor dataset & data loader
from torch.utils.data import TensorDataset, DataLoader

import torch.nn.functional as F

In [133]:
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [134]:
# Визначаємо dataset
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[0.],
         [1.],
         [1.]]))

8. Визначте data loader з класом **DataLoader** для підготовленого датасету `train_ds`, встановіть розмір батчу на 5 та увімкніть перемішування даних для ефективного навчання моделі. Виведіть перший елемент в дата лоадері.

In [135]:
# Визначаємо data loader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)
next(iter(train_dl))

[tensor([[ 69.,  96.,  70.],
         [ 87., 134.,  58.],
         [102.,  43.,  37.],
         [ 73.,  67.,  43.],
         [ 73.,  67.,  43.]]),
 tensor([[1.],
         [1.],
         [0.],
         [0.],
         [0.]])]

9. Створіть клас `LogReg` для логістичної регресії, наслідуючи модуль `torch.nn.Module` за прикладом в лекції (в частині про FeedForward мережі).

  У нас модель складається з лінійної комбінації вхідних значень і застосування фукнції сигмоїда. Тож, нейромережа буде складатись з лінійного шару `nn.Linear` і використання активації `nn.Sigmid`. У створеному класі мають бути реалізовані методи `__init__` з ініціалізацією шарів і метод `forward` для виконання прямого проходу моделі через лінійний шар і функцію активації.

  Створіть екземпляр класу `LogReg` в змінній `model`.

In [136]:
class LogReg(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear1 = nn.Linear(3, 1)
        self.act1 = nn.Sigmoid()

    def forward(self, x):
        x = self.linear1(x)
        x = self.act1(x)
        return x

In [137]:
model = LogReg()

10. Задайте оптимізатор `Stockastic Gradient Descent` в змінній `opt` для навчання моделі логістичної регресії. А також визначіть в змінній `loss` функцію втрат `binary_cross_entropy` з модуля `torch.nn.functional` для обчислення втрат моделі. Обчисліть втрати для поточних передбачень і міток, а потім виведіть їх. Зробіть висновок, чи моделі вдалось навчитись?

In [138]:
opt = torch.optim.SGD(model.parameters(), 1e-4)
loss_fn = F.binary_cross_entropy

In [139]:
loss = loss_fn(model(inputs), targets)
print(loss)

tensor(1.7988, grad_fn=<BinaryCrossEntropyBackward0>)


11. Візьміть з лекції функцію для тренування моделі з відстеженням значень втрат і навчіть щойно визначену модель на 1000 епохах. Виведіть після цього графік зміни loss, фінальні передбачення і значення таргетів.

In [140]:
# Модифікована функцію fit для відстеження втрат
def fit_return_loss(num_epochs, model, loss_fn, opt, train_dl):
    losses = []
    for epoch in range(num_epochs):
        # Ініціалізуємо акумулятор для втрат
        total_loss = 0

        for xb, yb in train_dl:
            # Генеруємо передбачення
            pred = model(xb)

            # Обчислюємо втрати
            loss = loss_fn(pred, yb)

            # Виконуємо градієнтний спуск
            loss.backward()
            opt.step()
            opt.zero_grad()

            # Накопичуємо втрати
            total_loss += loss.item()

        # Обчислюємо середні втрати для епохи
        avg_loss = total_loss / len(train_dl)
        losses.append(avg_loss)

        # Виводимо підсумок епохи
        if (epoch + 1) % 10 == 0:
          print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}')
    return losses

In [141]:
loss = fit_return_loss(1000, model, loss_fn, opt, train_dl)

Epoch [10/1000], Loss: 0.1108
Epoch [20/1000], Loss: 0.0702
Epoch [30/1000], Loss: 0.0698
Epoch [40/1000], Loss: 0.0696
Epoch [50/1000], Loss: 0.0709
Epoch [60/1000], Loss: 0.0708
Epoch [70/1000], Loss: 0.0691
Epoch [80/1000], Loss: 0.0689
Epoch [90/1000], Loss: 0.0682
Epoch [100/1000], Loss: 0.0685
Epoch [110/1000], Loss: 0.0683
Epoch [120/1000], Loss: 0.0681
Epoch [130/1000], Loss: 0.0675
Epoch [140/1000], Loss: 0.0688
Epoch [150/1000], Loss: 0.0676
Epoch [160/1000], Loss: 0.0674
Epoch [170/1000], Loss: 0.0683
Epoch [180/1000], Loss: 0.0670
Epoch [190/1000], Loss: 0.0669
Epoch [200/1000], Loss: 0.0667
Epoch [210/1000], Loss: 0.0679
Epoch [220/1000], Loss: 0.0673
Epoch [230/1000], Loss: 0.0662
Epoch [240/1000], Loss: 0.0675
Epoch [250/1000], Loss: 0.0659
Epoch [260/1000], Loss: 0.0671
Epoch [270/1000], Loss: 0.0666
Epoch [280/1000], Loss: 0.0654
Epoch [290/1000], Loss: 0.0652
Epoch [300/1000], Loss: 0.0664
Epoch [310/1000], Loss: 0.0658
Epoch [320/1000], Loss: 0.0656
Epoch [330/1000],

По цим епохамвидно, що потрохи падає лос. Якщо дивитись в одну епоху періодом 100, наприклад епоха 600 Loss: 0.0613 на початок нової епохи 700 Loss: 0.0590. Така тенденція по всім, лос зменшується